## Importação de Dados
- Visão Geral dos Dados
    * Diagrama
* Fontes
    - Postgres
    
- Group By
- Pivot
- Functions
- Exclude

In [ ]:
!pip install duckdb --quiet

In [7]:
import duckdb 
%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [8]:
%%sql 

INSTALL postgres;
LOAD postgres;
ATTACH 'dbname=lord_duckdb_postgres user=lord password=postgres123 host=127.0.0.1' AS db (TYPE POSTGRES);

SHOW ALL TABLES;

Running query in 'duckdb'

database,schema,name,column_names,column_types,temporary
db,public,aeronave,"['codigo_ocorrencia2', 'aeronave_matricula', 'aeronave_operador_categoria', 'aeronave_tipo_veiculo', 'aeronave_fabricante', 'aeronave_modelo', 'aeronave_tipo_icao', 'aeronave_motor_tipo', 'aeronave_motor_quantidade', 'aeronave_pmd', 'aeronave_pmd_categoria', 'aeronave_assentos', 'aeronave_ano_fabricacao', 'aeronave_pais_fabricante', 'aeronave_pais_registro', 'aeronave_registro_categoria', 'aeronave_registro_segmento', 'aeronave_voo_origem', 'aeronave_voo_destino', 'aeronave_fase_operacao', 'aeronave_tipo_operacao', 'aeronave_nivel_dano', 'aeronave_fatalidades_total']","['INTEGER', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'INTEGER', 'INTEGER', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'INTEGER']",False
db,public,big_fact_ocorrencias,"['codigo_ocorrencia', 'codigo_ocorrencia_tipo', 'ocorrencia_tipo', 'codigo_ocorrencia_tipo_categoria', 'ocorrencia_tipo_categoria', 'taxonomia_tipo_icao', 'codigo_ocorrencia_uf', 'ocorrencia_uf', 'codigo_ocorrencia_cidade', 'ocorrencia_cidade', 'codigo_ocorrencia_pais', 'ocorrencia_pais', 'codigo_ocorrencia_aerodromo', 'ocorrencia_aerodromo', 'codigo_investigacao_status', 'investigacao_status', 'codigo_investigacao_aeronave_liberada', 'investigacao_aeronave_liberada', 'codigo_divulgacao_relatorio_publicado', 'divulgacao_relatorio_publicado', 'codigo_ocorrencia_saida_pista', 'ocorrencia_saida_pista', 'codigo_aeronave_tipo_veiculo', 'aeronave_tipo_veiculo', 'codigo_aeronave_fabricante', 'aeronave_fabricante', 'codigo_aeronave_modelo', 'aeronave_modelo', 'codigo_aeronave_tipo_icao', 'aeronave_tipo_icao', 'codigo_aeronave_motor_tipo', 'aeronave_motor_tipo', 'codigo_aeronave_motor_quantidade', 'aeronave_motor_quantidade', 'codigo_aeronave_pmd', 'aeronave_pmd', 'codigo_aeronave_ano_fabricacao', 'aeronave_ano_fabricacao', 'codigo_aeronave_assentos', 'aeronave_assentos', 'codigo_aeronave_pais_fabricante', 'aeronave_pais_fabricante', 'codigo_aeronave_pais_registro', 'aeronave_pais_registro', 'codigo_aeronave_registro_categoria', 'aeronave_registro_categoria', 'codigo_aeronave_registro_segmento', 'aeronave_registro_segmento', 'codigo_aeronave_voo_origem', 'aeronave_voo_origem', 'codigo_aeronave_voo_destino', 'aeronave_voo_destino', 'codigo_aeronave_fase_operacao', 'aeronave_fase_operacao', 'codigo_aeronave_tipo_operacao', 'aeronave_tipo_operacao', 'codigo_aeronave_nivel_dano', 'aeronave_nivel_dano', 'codigo_fator_condicionante', 'fator_condicionante', 'codigo_fator_aspecto', 'fator_aspecto', 'codigo_fator_area', 'fator_area', 'codigo_recomendacao_destinatario_sigla', 'recomendacao_destinatario_sigla', 'codigo_recomendacao_destinatario', 'recomendacao_destinatario', 'recomendacao_conteudo']","['INTEGER', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'INTEGER', 'BIGINT', 'INTEGER', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'BIGINT', 'VARCHAR', 'VARCHAR']",False
db,public,calendario,"['data_completa', 'hora', 'minuto', 'dia', 'mes', 'ano', 'dia_da_semana', 'semana_do_ano', 'trimestre', 'e_feriado', 'e_fim_de_semana']","['DATE', 'INTEGER', 'INTEGER', 'INTEGER', 'INTEGER', 'INTEGER', 'VARCHAR', 'INTEGER', 'INTEGER', 'BOOLEAN', 'BOOLEAN']",False
db,public,dimensao_aeronave_ano_fabricacao,"['codigo_aeronave_ano_fabricacao', 'aeronave_ano_fabricacao']","['BIGINT', 

In [ ]:
%%sql

CALL postgres_execute('db', ' CREATE TABLE  RECOMENDACAO (
            codigo_ocorrencia4 INTEGER,
            recomendacao_numero varchar(2000),
            recomendacao_dia_assinatura DATE null,
            recomendacao_dia_encaminhamento DATE null,
            recomendacao_dia_feedback DATE null,
            recomendacao_status VARCHAR(2000),
            recomendacao_destinatario_sigla VARCHAR(2000),
            recomendacao_destinatario VARCHAR(2000),
            recomendacao_conteudo TEXT
        );');




In [ ]:
!pip install psycopg2 --quiet

In [ ]:
import csv
import psycopg2

# Connect to PostgreSQL database
conn = psycopg2.connect(
    dbname="lord_duckdb_postgres", 
    user="lord", 
    password="postgres123", 
    host="127.0.0.1", 
    port="5432"
)
cursor = conn.cursor()
cursor.execute("delete from recomendacao;")
# Open and read the CSV file
with open('data/aula5/recomendacao.csv', newline='', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=';')
    next(csvreader)  # Skip the header row
    for row in csvreader:
        # Replace "NULL" with None for date fields
        row = [None if value == "NULL" else value for value in row]
        #print(row)
        # Assuming the CSV columns match the table structure
        cursor.execute('''
            INSERT INTO recomendacao (
                codigo_ocorrencia4, recomendacao_numero, recomendacao_dia_assinatura, 
                recomendacao_dia_encaminhamento, recomendacao_dia_feedback, recomendacao_status, 
                recomendacao_destinatario_sigla, recomendacao_destinatario, recomendacao_conteudo
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        ''', row)

# Commit and close
conn.commit()
conn.close()

In [ ]:
%%sql 

CALL postgres_execute('db', ' CREATE TABLE IF NOT EXISTS OCORRENCIA (
            codigo_ocorrencia1 INTEGER,
            codigo_ocorrencia4 INTEGER,
            codigo_ocorrencia2 INTEGER,
            codigo_ocorrencia3 INTEGER,
            codigo_ocorrencia INTEGER,
            ocorrencia_classificacao VARCHAR(255),
            ocorrencia_longitude double precision,
            ocorrencia_latitude double precision,
            ocorrencia_cidade VARCHAR(255),
            ocorrencia_uf VARCHAR(255),
            ocorrencia_pais VARCHAR(255),
            ocorrencia_aerodromo VARCHAR(255),
            ocorrencia_dia DATE,
            ocorrencia_hora TIME null,
            investigacao_aeronave_liberada varchar(3),
            investigacao_status VARCHAR(255),
            divulgacao_relatorio_numero varchar(255),
            divulgacao_relatorio_publicado varchar(3),
            divulgacao_dia_publicacao DATE null,
            total_recomendacoes INTEGER,
            total_aeronaves_envolvidas varchar(3),
            ocorrencia_saida_pista VARCHAR(255)
        );');



In [ ]:
conn = psycopg2.connect(
    dbname="lord_duckdb_postgres", 
    user="lord", 
    password="postgres123", 
    host="127.0.0.1", 
    port="5432"
)

import re

def replace_dots_after_first(value):
    value = value.replace(',', '.')
    dots = [m.start() for m in re.finditer('\.', value)]
    
    # Se houver mais de um ponto, substitui todos os pontos após o primeiro por uma string vazia
    if len(dots) > 1:
        return value[:(dots[0]+1)]+value[(dots[0]+1):].replace('.', '')
    return value

def formatDate(value):
    if value == 'NULL':
        return None
    arr = value.split('/')
    return arr[2]+'-'+arr[1]+'-'+arr[0]

cursor = conn.cursor()
cursor.execute("delete from ocorrencia;")
# Open and read the CSV file
with open('data/aula5/ocorrencia.csv', newline='', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=';')
    next(csvreader)  # Skip the header row
    for row in csvreader:
        # Replace "NULL" with None for date fields
        #print(row) 
        if(row[6] != 'NULL' and row[6] != '' and '*' not in row[6]):
            row[6] = replace_dots_after_first(row[6]) 
        if(row[7] != 'NULL' and row[7] != '' and '*' not in row[7]): 
            row[7] = replace_dots_after_first(row[7]) 
        else:
            row[6] = None
            row[7] = None    

        row[12] = formatDate(row[12])
        row[18] = formatDate(row[18])
        row[13] = None if row[13] == 'NULL' else row[13]
        row[14] = None if row[14] == 'NULL' else row[14]
        try:
            cursor.execute('''
                INSERT INTO ocorrencia (
                    codigo_ocorrencia1, codigo_ocorrencia4, codigo_ocorrencia2, codigo_ocorrencia3, 
                    codigo_ocorrencia, ocorrencia_classificacao, ocorrencia_longitude, ocorrencia_latitude, 
                    ocorrencia_cidade, ocorrencia_uf, ocorrencia_pais, ocorrencia_aerodromo, 
                    ocorrencia_dia, ocorrencia_hora, investigacao_aeronave_liberada, investigacao_status, 
                    divulgacao_relatorio_numero, divulgacao_relatorio_publicado, divulgacao_dia_publicacao, 
                    total_recomendacoes, total_aeronaves_envolvidas, ocorrencia_saida_pista
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ''', row)
        except Exception as e:
            print(e)
            break
    
# Commit and close
conn.commit()
conn.close()

In [ ]:
%%sql 

CALL postgres_execute('db', 'CREATE TABLE IF NOT EXISTS OCORRENCIA_TIPO (
            codigo_ocorrencia1 INTEGER,
            ocorrencia_tipo VARCHAR(255),
            ocorrencia_tipo_categoria VARCHAR(255),
            taxonomia_tipo_icao VARCHAR(255)
        );');



In [ ]:
import csv
import psycopg2

# Connect to PostgreSQL database
conn = psycopg2.connect(
    dbname="lord_duckdb_postgres", 
    user="lord", 
    password="postgres123", 
    host="127.0.0.1", 
    port="5432"
)

cursor = conn.cursor()
cursor.execute("delete from ocorrencia_tipo;")
# Assuming you have a CSV file named 'ocorrencia_tipo.csv' with the appropriate columns
with open('data/aula5/ocorrencia_tipo.csv', newline='', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=';')  # Adjust delimiter if necessary
    next(csvreader)  # Skip the header row if your CSV has one
    for row in csvreader:
        try:
            cursor.execute('''
                INSERT INTO OCORRENCIA_TIPO (
                    codigo_ocorrencia1, ocorrencia_tipo, ocorrencia_tipo_categoria, taxonomia_tipo_icao
                ) VALUES (%s, %s, %s, %s)
            ''', row)
        except Exception as e:
            print(row)
            print(e)
            break
# Commit and close
conn.commit()
conn.close()

In [ ]:
%%sql 

CALL postgres_execute('db', ' CREATE TABLE IF NOT EXISTS AERONAVE (
            codigo_ocorrencia2 INTEGER,
            aeronave_matricula VARCHAR(255),
            aeronave_operador_categoria VARCHAR(255),
            aeronave_tipo_veiculo VARCHAR(255),
            aeronave_fabricante VARCHAR(255),
            aeronave_modelo VARCHAR(255),
            aeronave_tipo_icao VARCHAR(255),
            aeronave_motor_tipo VARCHAR(255),
            aeronave_motor_quantidade varchar(255),
            aeronave_pmd VARCHAR(255),
            aeronave_pmd_categoria VARCHAR(255),
            aeronave_assentos INTEGER null,
            aeronave_ano_fabricacao INTEGER null,
            aeronave_pais_fabricante VARCHAR(255),
            aeronave_pais_registro VARCHAR(255),
            aeronave_registro_categoria VARCHAR(255),
            aeronave_registro_segmento VARCHAR(255),
            aeronave_voo_origem VARCHAR(255),
            aeronave_voo_destino VARCHAR(255),
            aeronave_fase_operacao VARCHAR(255),
            aeronave_tipo_operacao VARCHAR(255),
            aeronave_nivel_dano VARCHAR(255),
            aeronave_fatalidades_total INTEGER
        );');



In [ ]:
import csv
import psycopg2

# Connect to PostgreSQL database
conn = psycopg2.connect(
    dbname="lord_duckdb_postgres", 
    user="lord", 
    password="postgres123", 
    host="127.0.0.1", 
    port="5432"
)
cursor = conn.cursor()
cursor.execute("delete from aeronave;")
# Assuming you have a CSV file named 'aeronave.csv' with the appropriate columns
with open('data/aula5/aeronave.csv', newline='', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=';')  # Adjust delimiter if necessary
    next(csvreader)  # Skip the header row if your CSV has one
    for row in csvreader:
        # Replace "NULL" with None for date fields
        row[11] = None if row[11] == 'NULL' else row[11]
        row[12] = None if row[12] == 'NULL' else row[12]
        cursor.execute('''
            INSERT INTO AERONAVE (
                codigo_ocorrencia2, aeronave_matricula, aeronave_operador_categoria, 
                aeronave_tipo_veiculo, aeronave_fabricante, aeronave_modelo, 
                aeronave_tipo_icao, aeronave_motor_tipo, aeronave_motor_quantidade, 
                aeronave_pmd, aeronave_pmd_categoria, aeronave_assentos, 
                aeronave_ano_fabricacao, aeronave_pais_fabricante, aeronave_pais_registro, 
                aeronave_registro_categoria, aeronave_registro_segmento, aeronave_voo_origem, 
                aeronave_voo_destino, aeronave_fase_operacao, aeronave_tipo_operacao, 
                aeronave_nivel_dano, aeronave_fatalidades_total
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ''', row)

# Commit and close
conn.commit()
conn.close()

In [ ]:
%%sql 

CALL postgres_execute('db', 'CREATE TABLE IF NOT EXISTS FATOR_CONTRIBUINTE (
            codigo_ocorrencia3 INTEGER,
            fator_nome VARCHAR(255),
            fator_aspecto VARCHAR(255),
            fator_condicionante VARCHAR(255),
            fator_area VARCHAR(255)
        );');

In [ ]:
import csv
import psycopg2

# Connect to PostgreSQL database
conn = psycopg2.connect(
    dbname="lord_duckdb_postgres", 
    user="lord", 
    password="postgres123", 
    host="127.0.0.1", 
    port="5432"
)


cursor = conn.cursor()
cursor.execute("delete from fator_contribuinte;")
# Assuming you have a CSV file named 'fator_contribuinte.csv' with the appropriate columns
with open('data/aula5/fator_contribuinte.csv', newline='', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=';')  # Adjust delimiter if necessary
    next(csvreader)  # Skip the header row if your CSV has one
    for row in csvreader:
        cursor.execute('''
            INSERT INTO FATOR_CONTRIBUINTE (
                codigo_ocorrencia3, fator_nome, fator_aspecto, fator_condicionante, fator_area
            ) VALUES (%s, %s, %s, %s, %s)
        ''', row)

# Commit and close
conn.commit()
conn.close()

In [6]:
%%sql

DETACH db;
INSTALL postgres;
LOAD postgres;
ATTACH 'dbname=lord_duckdb_postgres user=lord password=postgres123 host=127.0.0.1' AS db (TYPE POSTGRES);
SHOW ALL TABLES;

Running query in 'duckdb'

BinderException: Binder Error: Failed to detach database with name "db": database not found

### Detach para atualizar a listagem de tabelas no duckdb

## Exportação de Dados
- Visão Geral
- Saídas
    - S3 (minio/aws)
        - Power BI
        - Trino
        

    - Embedded Data Analysis 
        - Superset 
        
    - CSV
    - Json

In [ ]:
!pip install boto3 --quiet 

In [1]:
import boto3
import duckdb
conn = duckdb.connect('data/aula5/lord_duckdb.db') 
%load_ext sql
%sql conn --alias duckdb

# Create an S3 client on localhost:9000, using the access key and secret key
s3 = boto3.client('s3', endpoint_url='http://localhost:9000', aws_access_key_id='minio_access_key', aws_secret_access_key='minio_secret_key')

#get a lord-bucket
response = s3.list_buckets()
buckets = [bucket['Name'] for bucket in response['Buckets']]
print("Buckets:", buckets)


#create a lord-duckdb bucket if not exists
if 'lord-duckdb' not in buckets:
    s3.create_bucket(Bucket='lord-duckdb')


There's a new jupysql version available (0.10.12), you're running 0.10.10. To upgrade: pip install jupysql --upgrade
Deploy Dash apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup
Buckets: ['lord-duckdb']


In [ ]:
%%sql
select * from db.OCORRENCIA limit 5;

In [2]:
%%sql

INSTALL aws;
INSTALL httpfs;
LOAD aws;
LOAD httpfs;

Running query in 'duckdb'

Success


In [11]:
%%sql  


CREATE or replace persistent SECRET secret1 (
    TYPE S3, 
    KEY_ID 'minio_access_key',
    SECRET 'minio_secret_key',
    REGION 'us-east-1',
    URL_STYLE 'path', 
    USE_SSL 'false', 
    ENDPOINT '172.19.0.1:9000'
);

Running query in 'duckdb'

Success
True


In [ ]:
%%sql

--DROP TEMPORARY SECRET secret1;

FROM duckdb_secrets();

In [12]:
%%sql

select * from 's3://lord-duckdb/ocorrencia.csv';

Running query in 'duckdb'

codigo_ocorrencia1,codigo_ocorrencia4,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_longitude,ocorrencia_latitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
83097,83097,83097,83097,83097,ACIDENTE,-2.352777777777,-4.563944444444,PARAIBUNA,SP,BRASIL,None,2023-12-31,18:00:00,SIM,FINALIZADA,None,NÃO,None,0,1,NÃO
83085,83085,83085,83085,83085,INCIDENTE,-9.868333333333,-6.789805555555,RIO BRANCO,AC,BRASIL,SBRB,2023-12-13,11:33:00,SIM,FINALIZADA,None,NÃO,None,0,1,NÃO
83074,83074,83074,83074,83074,INCIDENTE,-1.962444444444,-4.397194444444,BELO HORIZONTE,MG,BRASIL,SBCF,2023-12-20,20:00:00,SIM,FINALIZADA,None,NÃO,None,0,1,NÃO
83063,83063,83063,83063,83063,INCIDENTE,-1.587111111111,-4.791861111111,BRASÍLIA,DF,BRASIL,SBBR,2023-12-29,21:20:00,SIM,FINALIZADA,None,NÃO,None,0,1,NÃO
83051,83051,83051,83051,83051,INCIDENTE,-4.871388888888,-668.975,CARAUARI,AM,BRASIL,SWCA,2023-12-09,18:30:00,SIM,FINALIZADA,None,NÃO,None,0,1,NÃO
83044,83044,83044,83044,83044,INCIDENTE,-2.300694444444,-4.713444444444,CAMPINAS,SP,BRASIL,SBKP,2023-12-29,23:50:00,SIM,FINALIZADA,None,NÃO,None,0,1,NÃO
83036,83036,83036,83036,83036,INCIDENTE,-2.292083333333,-4.207138888888,CABO FRIO,RJ,BRASIL,SBCB,2023-12-30,21:47:00,SIM,FINALIZADA,None,NÃO,None,0,1,NÃO
83035,83035,83035,83035,83035,INCIDENTE,-3.854722222222,-3.242833333333,FERNANDO DE NORONHA,PE,BRASIL,SBFN,2023-12-30,21:00:00,SIM,FINALIZADA,None,NÃO,None,0,1,NÃO
83033,83033,83033,83033,83033,INCIDENTE,-22.175,-5.142444444444,PRESIDENTE PRUDENTE,SP,BRASIL,SBDN,2023-12-28,10:34:00,SIM,FINALIZADA,None,NÃO,None,0,1,NÃO
83032,83032,83032,83032,83032,INCIDENTE,-10.29,-4.835777777777,PALMAS,TO,BRASIL,SBPJ,2023-12-29,05:07:00,SIM,FINALIZADA,None,NÃO,None,0,1,NÃO


In [13]:
conn.execute('''  

copy (select * from db.OCORRENCIA) to 's3://lord-duckdb/ocorrencia.parquet' with (FORMAT PARQUET);''')

In [ ]:
%%sql

CREATE  or replace TABLE Fato_Ocorrencia (
	codigo_ocorrencia INT ,
	ocorrencia_dia DATE,
	ocorrencia_hora TIME,
	ocorrencia_tipo VARCHAR(255),
	ocorrencia_cidade VARCHAR(255),
	ocorrencia_uf CHAR(2),
	ocorrencia_pais VARCHAR(255),
	ocorrencia_aerodromo VARCHAR(4),
	ocorrencia_saida_pista varchar(3),
	total_recomendacoes INT,
	total_aeronaves_envolvidas INT 
);

CREATE  or replace TABLE Fato_Fator_Contribuinte (
	codigo_ocorrencia INT,
	fator_nome VARCHAR(255),
	fator_aspecto VARCHAR(255),
	fator_condicionante BOOLEAN,
	fator_area VARCHAR(255),
	PRIMARY KEY (codigo_ocorrencia, fator_nome)
);

CREATE  or replace TABLE Fato_Investigacao (
	codigo_ocorrencia INT PRIMARY KEY,
	investigacao_aeronave_liberada BOOLEAN,
	investigacao_status VARCHAR(255)
);

CREATE  or replace TABLE Fato_Divulgacao (
	codigo_ocorrencia INT PRIMARY KEY,
	divulgacao_relatorio_numero VARCHAR(255),
	divulgacao_relatorio_publicado BOOLEAN,
	divulgacao_dia_publicacao DATE
);

CREATE or replace TABLE Dimensao_Aeronave (
	codigo_aeronave INT PRIMARY KEY,
	aeronave_matricula VARCHAR(255),
	aeronave_tipo_veiculo VARCHAR(255),
	aeronave_fabricante VARCHAR(255),
	aeronave_modelo VARCHAR(255),
	aeronave_tipo_cao VARCHAR(255),
	aeronave_motor_tipo VARCHAR(255),
	aeronave_motor_quantidade INT,
	aeronave_pmd BOOLEAN,
	aeronave_pmd_categoria VARCHAR(255),
	aeronave_assentos INT,
	aeronave_ano_fabricacao integer,
	aeronave_pais_fabricante VARCHAR(255),
	aeronave_pais_registro VARCHAR(255),
	aeronave_registro_categoria VARCHAR(255),
	aeronave_registro_segmento VARCHAR(255),
	aeronave_voo_origem VARCHAR(255),
	aeronave_voo_destino VARCHAR(255),
	aeronave_fase_operacao VARCHAR(255),
	aeronave_tipo_operacao VARCHAR(255)
);

In [ ]:
%%sql

select distinct ocorrencia_tipo from db.ocorrencia_tipo  where ocorrencia_tipo ilike '%a%' 

In [ ]:
%%sql


delete from Fato_Ocorrencia;
insert into Fato_Ocorrencia 
select o.codigo_ocorrencia,
o.ocorrencia_dia,
o.ocorrencia_hora,
ot.ocorrencia_tipo,
o.ocorrencia_cidade,
o.ocorrencia_uf,
o.ocorrencia_pais,
o.ocorrencia_aerodromo,
o.ocorrencia_saida_pista,
count(r.*) as total_recomendacoes,
count(a.*) as total_aeronaves_envolvidas
  from db.ocorrencia o 
left join db.ocorrencia_tipo ot on o.codigo_ocorrencia1 = ot.codigo_ocorrencia1
left join db.aeronave a on o.codigo_ocorrencia2 = a.codigo_ocorrencia2
left join db.fator_contribuinte fc on o.codigo_ocorrencia3 = fc.codigo_ocorrencia3
left join db.recomendacao r on o.codigo_ocorrencia4 = r.codigo_ocorrencia4
group by all

In [22]:
%%sql
copy ( select name from (SHOW ALL TABLES) order by 1) to 's3://lord-duckdb/tabelas.csv' with (FORMAT CSV);

Running query in 'duckdb'

Count
38


In [27]:
%%sql

copy ( select * from db.dimensao_aeronave_ano_fabricacao        ) to 's3://lord-duckdb/bi/dimensao_aeronave_ano_fabricacao.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_aeronave_assentos              ) to 's3://lord-duckdb/bi/dimensao_aeronave_assentos.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_aeronave_fase_operacao         ) to 's3://lord-duckdb/bi/dimensao_aeronave_fase_operacao.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_aeronave_modelo                ) to 's3://lord-duckdb/bi/dimensao_aeronave_modelo.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_aeronave_motor_quantidade      ) to 's3://lord-duckdb/bi/dimensao_aeronave_motor_quantidade.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_aeronave_motor_tipo            ) to 's3://lord-duckdb/bi/dimensao_aeronave_nivel_dano.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_aeronave_nivel_dano            ) to 's3://lord-duckdb/bi/dimensao_aeronave_nivel_dano.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_aeronave_pais_fabricante       ) to 's3://lord-duckdb/bi/dimensao_aeronave_pais_fabricante.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_aeronave_pais_registro         ) to 's3://lord-duckdb/bi/dimensao_aeronave_pais_registro.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_aeronave_pmd                   ) to 's3://lord-duckdb/bi/dimensao_aeronave_pmd.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_aeronave_registro_categoria    ) to 's3://lord-duckdb/bi/dimensao_aeronave_registro_segmento.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_aeronave_registro_segmento     ) to 's3://lord-duckdb/bi/dimensao_aeronave_registro_segmento.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_aeronave_tipo_icao             ) to 's3://lord-duckdb/bi/dimensao_aeronave_tipo_operacao.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_aeronave_tipo_operacao         ) to 's3://lord-duckdb/bi/dimensao_aeronave_tipo_operacao.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_aeronave_tipo_veiculo          ) to 's3://lord-duckdb/bi/dimensao_aeronave_tipo_veiculo.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_aeronave_voo_destino           ) to 's3://lord-duckdb/bi/dimensao_codigo_aeronave_motor_tipo.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_aeronave_voo_origem            ) to 's3://lord-duckdb/bi/dimensao_codigo_aeronave_motor_tipo.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_codigo_aeronave_motor_tipo     ) to 's3://lord-duckdb/bi/dimensao_codigo_aeronave_motor_tipo.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_divulgacao_relatorio_publicado ) to 's3://lord-duckdb/bi/dimensao_divulgacao_relatorio_publicado.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_fator_area                     ) to 's3://lord-duckdb/bi/dimensao_fator_area.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_fator_aspecto                  ) to 's3://lord-duckdb/bi/dimensao_fator_aspecto.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_fator_condicionante            ) to 's3://lord-duckdb/bi/dimensao_fator_condicionante.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_investigacao_aeronave_liberada ) to 's3://lord-duckdb/bi/dimensao_investigacao_aeronave_liberada.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_investigacao_status            ) to 's3://lord-duckdb/bi/dimensao_investigacao_status.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_ocorrencia_aerodromo           ) to 's3://lord-duckdb/bi/dimensao_ocorrencia_aerodromo.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_ocorrencia_cidade              ) to 's3://lord-duckdb/bi/dimensao_ocorrencia_pais.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_ocorrencia_pais                ) to 's3://lord-duckdb/bi/dimensao_ocorrencia_pais.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_ocorrencia_saida_pista         ) to 's3://lord-duckdb/bi/dimensao_ocorrencia_saida_pista.csv' with (FORMAT CSV);
copy ( select * from db.dimensao_ocorrencia_uf                  ) to 's3://lord-duckdb/bi/dimensao_ocorrencia_uf.csv' with (FORMAT CSV);
copy ( select * from db.dimension_ocorrencia_tipo               ) to 's3://lord-duckdb/bi/dimension_ocorrencia_tipo.csv' with (FORMAT CSV);
copy ( select * from db.dimension_ocorrencia_tipo_categoria     ) to 's3://lord-duckdb/bi/dimension_ocorrencia_tipo_categoria.csv' with (FORMAT CSV);
copy ( select * from db.big_fact_ocorrencias     ) to 's3://lord-duckdb/bi/big_fact_ocorrencias.csv' with (FORMAT CSV);






Running query in 'duckdb'

Count
33090


In [29]:
%%sql

create view dimensao_aeronave_ano_fabricacao        as select * from 's3://lord-duckdb/bi/dimensao_aeronave_ano_fabricacao.csv';
create view dimensao_aeronave_assentos              as select * from 's3://lord-duckdb/bi/dimensao_aeronave_assentos.csv';
create view dimensao_aeronave_fase_operacao         as select * from 's3://lord-duckdb/bi/dimensao_aeronave_fase_operacao.csv';
create view dimensao_aeronave_modelo                as select * from 's3://lord-duckdb/bi/dimensao_aeronave_modelo.csv';
create view dimensao_aeronave_motor_quantidade      as select * from 's3://lord-duckdb/bi/dimensao_aeronave_motor_quantidade.csv';
create view dimensao_aeronave_motor_tipo            as select * from 's3://lord-duckdb/bi/dimensao_aeronave_nivel_dano.csv';
create view dimensao_aeronave_nivel_dano            as select * from 's3://lord-duckdb/bi/dimensao_aeronave_nivel_dano.csv';
create view dimensao_aeronave_pais_fabricante       as select * from 's3://lord-duckdb/bi/dimensao_aeronave_pais_fabricante.csv';
create view dimensao_aeronave_pais_registro         as select * from 's3://lord-duckdb/bi/dimensao_aeronave_pais_registro.csv';
create view dimensao_aeronave_pmd                   as select * from 's3://lord-duckdb/bi/dimensao_aeronave_pmd.csv';
create view dimensao_aeronave_registro_categoria    as select * from 's3://lord-duckdb/bi/dimensao_aeronave_registro_segmento.csv';
create view dimensao_aeronave_registro_segmento     as select * from 's3://lord-duckdb/bi/dimensao_aeronave_registro_segmento.csv';
create view dimensao_aeronave_tipo_icao             as select * from 's3://lord-duckdb/bi/dimensao_aeronave_tipo_operacao.csv';
create view dimensao_aeronave_tipo_operacao         as select * from 's3://lord-duckdb/bi/dimensao_aeronave_tipo_operacao.csv';
create view dimensao_aeronave_tipo_veiculo          as select * from 's3://lord-duckdb/bi/dimensao_aeronave_tipo_veiculo.csv';
create view dimensao_aeronave_voo_destino           as select * from 's3://lord-duckdb/bi/dimensao_codigo_aeronave_motor_tipo.csv';
create view dimensao_aeronave_voo_origem            as select * from 's3://lord-duckdb/bi/dimensao_codigo_aeronave_motor_tipo.csv';
create view dimensao_codigo_aeronave_motor_tipo     as select * from 's3://lord-duckdb/bi/dimensao_codigo_aeronave_motor_tipo.csv';
create view dimensao_divulgacao_relatorio_publicado as select * from 's3://lord-duckdb/bi/dimensao_divulgacao_relatorio_publicado.csv';
create view dimensao_fator_area                     as select * from 's3://lord-duckdb/bi/dimensao_fator_area.csv';
create view dimensao_fator_aspecto                  as select * from 's3://lord-duckdb/bi/dimensao_fator_aspecto.csv';
create view dimensao_fator_condicionante            as select * from 's3://lord-duckdb/bi/dimensao_fator_condicionante.csv';
create view dimensao_investigacao_aeronave_liberada as select * from 's3://lord-duckdb/bi/dimensao_investigacao_aeronave_liberada.csv';
create view dimensao_investigacao_status            as select * from 's3://lord-duckdb/bi/dimensao_investigacao_status.csv';
create view dimensao_ocorrencia_aerodromo           as select * from 's3://lord-duckdb/bi/dimensao_ocorrencia_aerodromo.csv';
create view dimensao_ocorrencia_cidade              as select * from 's3://lord-duckdb/bi/dimensao_ocorrencia_pais.csv';
create view dimensao_ocorrencia_pais                as select * from 's3://lord-duckdb/bi/dimensao_ocorrencia_pais.csv';
create view dimensao_ocorrencia_saida_pista         as select * from 's3://lord-duckdb/bi/dimensao_ocorrencia_saida_pista.csv';
create view dimensao_ocorrencia_uf                  as select * from 's3://lord-duckdb/bi/dimensao_ocorrencia_uf.csv';
create view dimension_ocorrencia_tipo               as select * from 's3://lord-duckdb/bi/dimension_ocorrencia_tipo.csv';
create view dimension_ocorrencia_tipo_categoria     as select * from 's3://lord-duckdb/bi/dimension_ocorrencia_tipo_categoria.csv';


Running query in 'duckdb'

Count


In [32]:
%%sql 

show tables

Running query in 'duckdb'

name
dimensao_aeronave_ano_fabricacao
dimensao_aeronave_assentos
dimensao_aeronave_fase_operacao
dimensao_aeronave_modelo
dimensao_aeronave_motor_quantidade
dimensao_aeronave_motor_tipo
dimensao_aeronave_nivel_dano
dimensao_aeronave_pais_fabricante
dimensao_aeronave_pais_registro
dimensao_aeronave_pmd
